**Download Data**\
The granular tick/trade data from DataBento can be downloaded from:
https://drive.google.com/file/d/1WE4YTNmtWPSvEsYBDD_V2lUYEE_J_sMJ/view?usp=sharing

It should be saved to the `data_dir` directory specified in the "Trade Strategy Variables" section below.

In [1]:
import pandas as pd


Below are variables for requesting data sets via DataBento API. Please use the example data from above instead of pulling from the API (there is a $ cost associated). These may not be needed once a database is established. We only want to pull the data once, when possible. Currently, these are only used in the code below for generating dynamic file names for outputting test data.

In [2]:
# Data Set Variables for DataBento; these may not be needed once a database is established
dataset = "GLBX.MDP3"
symbol = "NQZ3"
schema = "trades"
start_date = "2023-12-01T00:00:00"
end_date = "2023-12-31T23:59:59"
stype_in = "raw_symbol"


**Trade Strategy Variables**

Currently, these variables only apply to this specific strategy. They would need to be made customizable and toggleable in the future when developing new strategies. See code comments for individual variable descriptions.


**Data Source(s) and Storage**

These can be changed as needed. `data_dir` is the source data and `temp_data_dir` is used throughout the code below to store temporary output for debugging (the same output you see in the Jupyter notebook cells).

In [3]:
# Trade Strategy Variables
trade_tf = "5min" # The timeframe the trade is framed on. Use 'min' instead of 'm' for minutes because of df.resample() below
entry_tf = "30s" # The timeframe the trade is executed on
data_open_hour = 4 # The hour to begin the data feed for the trading day. Likely earlier than the trade window open in order to identify the earliest setups. (24-hour)
data_close_hour = 17 # The hour to end the data feed for the trading day. At minimum, the hour that all trades should be closed for the day. (24-hour)
tw_open_hour = 8 # The hour that the active trade window opens (24-hour)
tw_close_hour = 16 # The hour that the active trade window closes (24-hour)
stop_loss = 12 # Stop Loss (in points)
profit_target = 10 # Profit Target (in points)

# Data Source(s) and Storage
data_dir = "./data" # Where the original granular data is stored locally
temp_data_dir = "./data/temp" # Directory to store temp data for verification purposes


**Risk Management Variables**

These will typically be used for backtesting the majority of strategies. These may change based on the python backtesting library we end up using. Descriptions for each can be found in the code comments.

In [4]:
# Position Size and Risk Management Variables
starting_balance = 3000 # Starting account balance for simulation
scaling_interval = 5000 # Interval amount to add `scaling_position_size` from below
position_size = 1 # How many contracts/shares of the symbol
scaling_position_size = 1 # How many contracts/shares to add based on `scaling_interval` above
stop_loss = 12.00 # Number of total points for static stop loss
profit_target = 10.00 # Number of total points for static take profit target

Load test data into a pandas dataframe, convert time to US/Eastern, and set the index to `ts_recv` which is the time that each trade is finally executed on the exchange.

In [5]:
# Create a dataframe from DataBento .csv file for the most granular data
# **This will need changed to a database connection in the future**
df = pd.read_csv(f"{data_dir}/{symbol}_DataBento_{schema}_Dec_2023.csv")

# Format and convert to US Eastern time
# "ts_recv" is the sum of "ts_event" plus "ts_in_delta" in case it is not already calculated in the data
df["ts_recv"] = pd.to_datetime(df["ts_recv"], utc=True, dayfirst=False).dt.tz_convert("America/New_York")

# Set the index column
df.set_index("ts_recv", inplace=True)

# Format and output the original, granular dataframe after indexing
with pd.option_context("display.min_rows", 10, "display.max_columns", None, "display.width", 1000):
    print(df)

FileNotFoundError: [Errno 2] No such file or directory: '../data/NQ/tick/2023/Dec/NQZ3_DataBento_trades_Dec_2023.csv'

Resampling the dataframe above into 30-second and 5-minute Open, High, Low, and Close data points (OHLC). These 4 data points (OHLC) make up a candlestick on a trading chart.

The data being output in this code block is also being saved to the `temp_data_dir` for easier browsing of the data for verification and troubleshooting.

<span style="color:yellow">***The resampled data generated directly below is manually verified to be correct and accurate***</span>

In [6]:
# Resample granular data into aggregated intervals for the strategy
# -------------------------------
# 30-second candlesticks (OHLC)
df_30s = df.resample("30s")["price"].ohlc().ffill()

# Format and display the resampled 30-second output
with pd.option_context("display.min_rows", 10, "display.max_columns", None, "display.width", 1000):
    print(f"\n######################\n30-sec resampled data:\n######################\n {df_30s}")
    
# Save resampled 30-second data to csv for verification purposes, if needed
df_30s.to_csv(f"{temp_data_dir}/{symbol}_30s_aggregated_ohlc.csv")
print(f"30-second aggregated data saved to: '{temp_data_dir}/{symbol}_30s_aggregated_ohlc.csv'")


# 5-minute candlesticks (OHLC)
df_5m = df.resample("5min")["price"].ohlc().ffill()

# Format and display the resampled 5-minute output
with pd.option_context("display.min_rows", 10, "display.max_columns", None, "display.width", 1000):
    print(f"\n#####################\n5-min resampled data:\n#####################\n {df_5m}")
    
# Save resampled 5-minute data to csv for verification purposes, if needed
df_5m.to_csv(f"{temp_data_dir}/{symbol}_5m_aggregated_ohlc.csv")
print(f"5-minute aggregated data saved to: '{temp_data_dir}/{symbol}_5min_aggregated_ohlc.csv'")

# 15-minute candlesticks (OHLC)
df_15m = df.resample("15min")["price"].ohlc().ffill()

# Format and display the resampled 5-minute output
with pd.option_context("display.min_rows", 10, "display.max_columns", None, "display.width", 1000):
    print(f"\n#####################\n5-min resampled data:\n#####################\n {df_15m}")
    
# Save resampled 5-minute data to csv for verification purposes, if needed
df_15m.to_csv(f"{temp_data_dir}/{symbol}_15m_aggregated_ohlc.csv")
print(f"15-minute aggregated data saved to: '{temp_data_dir}/{symbol}_15min_aggregated_ohlc.csv'")



######################
30-sec resampled data:
######################
                                open      high       low     close
ts_recv                                                          
2023-11-30 19:00:00-05:00  15964.25  15964.25  15962.25  15962.50
2023-11-30 19:00:30-05:00  15963.00  15963.75  15962.75  15963.75
2023-11-30 19:01:00-05:00  15963.50  15963.50  15963.00  15963.25
2023-11-30 19:01:30-05:00  15963.00  15963.25  15963.00  15963.25
2023-11-30 19:02:00-05:00  15962.75  15962.75  15961.00  15961.50
...                             ...       ...       ...       ...
2023-12-15 09:27:30-05:00  16580.50  16597.00  16575.00  16580.50
2023-12-15 09:28:00-05:00  16580.50  16589.00  16545.00  16545.00
2023-12-15 09:28:30-05:00  16582.00  16582.75  16544.75  16559.25
2023-12-15 09:29:00-05:00  16552.50  16582.00  16521.50  16557.75
2023-12-15 09:29:30-05:00  16554.00  16568.50  16533.00  16558.00

[42060 rows x 4 columns]
30-second aggregated data saved to: '../strat

Here, we are applying the defined data window variables from the strategy rules to the previously resampled data so that only the data within the desired data window is analyzed. There may be a much better way of doing this, I'm not sure.


The reason there is a data_window and trading_window is because the data needs to be analyzed prior to the trading window in case there is a valid trade entry as soon as the trading window opens (it happens quite often). For example, the trading strategy begins trading at 8AM. There may be a setup that allows for an entry at exactly 8AM. If the data window was the same as the trading window, that trade would be missed because it would not start looking for new swing highs and lows until after 8AM.


Perhaps just applying this time window to the original dataframe ***before*** resampling into the 30-sec and 5-min data is likely better and faster/more efficient.


<span style="color:yellow">***The filtered data generated directly below is manually verified to be correct and accurate***</span>

In [7]:
# Define the data window. In many cases, this will need to be larger than the active trading window to be sure that the earliest setups are identified.
def is_data_window(row):
    if isinstance(row.name, pd.Timestamp):
        return row.name.hour >= data_open_hour and row.name.hour < data_close_hour and row.name.dayofweek <= 4
    elif isinstance(row.name, tuple):
        return row.name[1].hour >= data_open_hour and row.name[1].hour < data_close_hour and row.name[1].dayofweek <= 4
    else:
        raise ValueError("Unexpected data type for row.name")

# Apply 'is_data_window' filter to granular data
df["is_data_window"] = df.apply(is_data_window, axis=1)

# Filter (copy) the results and only display rows where 'is_data_window' is True
df = df[df["is_data_window"]].copy()

# Display the filtered granular output for verification
with pd.option_context("display.min_rows", 10, "display.max_columns", None, "display.width", 1000):
    print(f"Applying 'is_data_window' to granular DataFrame and filtering results:\n {df}")

# Apply 'is_data_window' filter to 30-second resampled data
df_30s["is_data_window"] = df_30s.apply(is_data_window, axis=1)

# Filter (copy) the results and only display rows where 'is_data_window' is True
df_30s = df_30s[df_30s["is_data_window"]].copy()

# Display the filtered 30-second resampled output for verification
with pd.option_context("display.min_rows", 10, "display.max_columns", None, "display.width", 1000):
    print(f"Applying 'is_data_window' to 30-second resampled DataFrame and filtering results:\n {df_30s}")

# Apply 'is_data_window' filter to 5-minute resampled data   
df_5m["is_data_window"] = df_5m.apply(is_data_window, axis=1)

# Filter (copy) the results and only display rows where 'is_data_window' is True
df_5m = df_5m[df_5m["is_data_window"]].copy()

# Display the filtered 5-minute resampled output for verification
with pd.option_context("display.min_rows", 10, "display.max_columns", None, "display.width", 1000):
    print(f"Applying 'is_data_window' to 5-minute resampled DataFrame and filtering results:\n {df_5m}")


Applying 'is_data_window' to granular DataFrame and filtering results:
                                                                 ts_event  rtype  publisher_id  instrument_id action side  depth     price  size  flags  ts_in_delta   sequence symbol  is_data_window
ts_recv                                                                                                                                                                                              
2023-12-01 04:00:00.007095156-05:00  2023-12-01 09:00:00.006874529+00:00      0             1         260937      T    A      0  15981.25     1      0        15470  181627916   NQZ3            True
2023-12-01 04:00:00.030049105-05:00  2023-12-01 09:00:00.029849545+00:00      0             1         260937      T    B      0  15981.75     1      0        15192  181627987   NQZ3            True
2023-12-01 04:00:00.030548991-05:00  2023-12-01 09:00:00.030358007+00:00      0             1         260937      T    B      0  15981.7

At this point, all of the data is available in the necessary aggregated timeframes for the trading strategy and we can begin to look for Swing High and Swing Low candlestick patterns within the 5-minute OHLC candlestick data.


The code below identifies all swing highs and lows and only fills the most recent swing that was generated (whether or not it was broken doesn't matter here).


A "swing high" is a three candlestick pattern where the 1st and 3rd candle highs are lower (and not equal to) the high of the 2nd candle. A "swing low" is a three candlestick pattern where the 1st and 3rd candle lows are higher (and not equal to) the low of the 2nd candle.

<span style="color:yellow">***The data output directly below is manually verified to be correct and accurate***</span>

In [8]:
# Calculate the rolling swing highs and swing lows over the last three, 5-minute candlesticks
df_5m["SwingHigh"] = df_5m["high"].shift(2).where((df_5m["high"].shift(2) > df_5m["high"].shift(1)) & (df_5m["high"].shift(2) > df_5m["high"].shift(3))).shift(-1)
df_5m["SwingLow"] = df_5m["low"].shift(2).where((df_5m["low"].shift(2) < df_5m["low"].shift(1)) & (df_5m["low"].shift(2) < df_5m["low"].shift(3))).shift(-1)

# # Forward-fill NaNs in df_5m
# df_5m["SwingHigh"] = df_5m["SwingHigh"].ffill()
# df_5m["SwingLow"] = df_5m["SwingLow"].ffill()

with pd.option_context("display.min_rows", 20, "display.max_columns", None, "display.width", 1000):
    print(df_5m)


                               open      high       low     close  is_data_window  SwingHigh  SwingLow
ts_recv                                                                                               
2023-12-01 04:00:00-05:00  15981.25  15985.00  15980.50  15981.75            True        NaN       NaN
2023-12-01 04:05:00-05:00  15982.00  15987.00  15978.00  15985.25            True        NaN       NaN
2023-12-01 04:10:00-05:00  15985.50  15988.75  15982.75  15988.00            True        NaN  15978.00
2023-12-01 04:15:00-05:00  15987.75  15992.25  15986.50  15991.25            True        NaN       NaN
2023-12-01 04:20:00-05:00  15991.25  15994.50  15990.50  15992.25            True        NaN       NaN
2023-12-01 04:25:00-05:00  15992.25  15994.25  15988.75  15993.75            True   15994.50       NaN
2023-12-01 04:30:00-05:00  15993.50  16000.75  15993.25  15997.50            True        NaN  15988.75
2023-12-01 04:35:00-05:00  15997.50  15999.00  15993.25  15995.00        

Once the most recent swing highs and lows within the 5-minute data are being identified, we are waiting for price to move above the most recent swing high (`BreakHigh`) or below the most recent swing low (`BreakLow`).

**TO-DO:**
  - If `BreakHigh` or `BreakLow` are 'True', then reset `SwingHigh` and/or `SwingLow` to the next most recent swing high or low that has not been broken (currently stuck on this part here a little bit)

<span style="color:red">**PLEASE NOTE:** This output has not yet been verified to ensure that it's correct and valid.</span>

In [9]:
# Check for the "break" of the most recent swing high and/or swing low. For example, price needs to move above a swing high or below a swing low and cannot be equal to.
df_5m['BreakHigh'] = (df_5m['high'] > df_5m['SwingHigh'])
df_5m['BreakLow'] = (df_5m['low'] < df_5m['SwingLow'])

with pd.option_context("display.min_rows", 10, "display.max_columns", None, "display.width", 1000):
    print(df_5m)
    
# If `BreakHigh` or `BreakLow` are 'True', then reset `SwingHigh` and/or `SwingLow` to the next most recent swing high or low that has not been broken

                               open      high       low     close  is_data_window  SwingHigh  SwingLow  BreakHigh  BreakLow
ts_recv                                                                                                                    
2023-12-01 04:00:00-05:00  15981.25  15985.00  15980.50  15981.75            True        NaN       NaN      False     False
2023-12-01 04:05:00-05:00  15982.00  15987.00  15978.00  15985.25            True        NaN       NaN      False     False
2023-12-01 04:10:00-05:00  15985.50  15988.75  15982.75  15988.00            True        NaN   15978.0      False     False
2023-12-01 04:15:00-05:00  15987.75  15992.25  15986.50  15991.25            True        NaN       NaN      False     False
2023-12-01 04:20:00-05:00  15991.25  15994.50  15990.50  15992.25            True        NaN       NaN      False     False
...                             ...       ...       ...       ...             ...        ...       ...        ...       ...
2023-12-

Then, once a swing high or low has been broken, we check for the next candle to close ***in the opposite direction*** of the swing high or low that was broken.

For example, if a swing high was broken (price moved higher than the swing high), it will wait for a red (or down-close) candle (close < open) to close in the 5-minute data and return `true` in the dataframe for `DownClose`. If a swing low was broken (price moved lower than the swing low), it will wait for a green (or up-close) candle (close > open) to close in the 5-minute data and return `true` in the dataframe for `UpClose`.

<span style="color:red">**PLEASE NOTE:** This output has not yet been verified to ensure that it's correct and valid.</span>

***This appears to be correctly identifying when a swing high or low is broken and when an opposite close candle happens, but we need to update the dataframe as these columns become invalidated. Currently stuck on this part a bit.***

In [10]:
# Check for a candle to close in the opposite direction of the broken swing high/low
df_5m['DownClose'] = (df_5m['close'] < df_5m['open']) & df_5m['BreakHigh'].shift(1)
df_5m['UpClose'] = (df_5m['close'] > df_5m['open']) & df_5m['BreakLow'].shift(1)

with pd.option_context("display.min_rows", 10, "display.max_columns", None, "display.width", 1000):
    print(df_5m)

                               open      high       low     close  is_data_window  SwingHigh  SwingLow  BreakHigh  BreakLow  DownClose  UpClose
ts_recv                                                                                                                                        
2023-12-01 04:00:00-05:00  15981.25  15985.00  15980.50  15981.75            True        NaN       NaN      False     False      False    False
2023-12-01 04:05:00-05:00  15982.00  15987.00  15978.00  15985.25            True        NaN       NaN      False     False      False    False
2023-12-01 04:10:00-05:00  15985.50  15988.75  15982.75  15988.00            True        NaN   15978.0      False     False      False    False
2023-12-01 04:15:00-05:00  15987.75  15992.25  15986.50  15991.25            True        NaN       NaN      False     False      False    False
2023-12-01 04:20:00-05:00  15991.25  15994.50  15990.50  15992.25            True        NaN       NaN      False     False      False  

Here, we are applying the defined trading window variables from the strategy rules to the previously resampled data so that only the data within the desired trade window is analyzed. I'm sure there is likely a much better way of doing this and the data_window block further above.


The reason there is a data_window and trading_window is because the data needs to be analyzed prior to the trading window in case there is a valid trade entry as soon as the trading window opens (it happens quite often). For example, the trading strategy begins trading at 8AM. There may be a setup that allows for an entry at exactly 8AM. If the data window was the same as the trading window, that trade would be missed because it would not start looking for new swing highs and lows until after 8AM.


Perhaps just applying this time window to the original dataframe ***before*** resampling into the 30-sec and 5-min data is likely better and faster/more efficient.

<span style="color:red">**PLEASE NOTE:** This output has not yet been verified to ensure that it's correct and valid.</span>

In [11]:
# Define the trading window to filter data; this may move into a separated functions file in the future as it would likely be used often between strategies
def is_trading_window(row):
    if isinstance(row.name, pd.Timestamp):
        return row.name.hour >= tw_open_hour and row.name.hour < tw_close_hour and row.name.dayofweek <= 4
    elif isinstance(row.name, tuple):
        return row.name[1].hour >= tw_open_hour and row.name[1].hour < tw_close_hour and row.name[1].dayofweek <= 4
    else:
        raise ValueError("Unexpected data type for row.name")

# Apply 'is_trading_window' filter to granular data
df["is_trading_window"] = df.apply(is_trading_window, axis=1)
# Filter (copy) the results and only display rows where 'is_trading_window' is True
df = df[df["is_trading_window"]].copy()
# Display the filtered granular output for verification
with pd.option_context("display.min_rows", 10, "display.max_columns", None, "display.width", 1000):
    print(f"Applying 'is_trading_window' to granular DataFrame and filtering results:\n {df}")

# Apply 'is_trading_window' filter to 30-second resampled data
df_30s["is_trading_window"] = df_30s.apply(is_trading_window, axis=1)
# Filter (copy) the results and only display rows where 'is_trading_window' is True
df_30s = df_30s[df_30s["is_trading_window"]].copy()
# Display the filtered 30-second resampled output for verification
with pd.option_context("display.min_rows", 10, "display.max_columns", None, "display.width", 1000):
    print(f"Applying 'is_trading_window' to 30-second resampled DataFrame and filtering results:\n {df_30s}")

# Apply 'is_trading_window' filter to 5-minute resampled data   
df_5m["is_trading_window"] = df_5m.apply(is_trading_window, axis=1)
# Filter (copy) the results and only display rows where 'is_trading_window' is True
df_5m = df_5m[df_5m["is_trading_window"]].copy()
# Display the filtered 5-minute resampled output for verification
with pd.option_context("display.min_rows", 10, "display.max_columns", None, "display.width", 1000):
    print(f"Applying 'is_trading_window' to 5-minute resampled DataFrame and filtering results:\n {df_5m}")


Applying 'is_trading_window' to granular DataFrame and filtering results:
                                                                 ts_event  rtype  publisher_id  instrument_id action side  depth     price  size  flags  ts_in_delta   sequence symbol  is_data_window  is_trading_window
ts_recv                                                                                                                                                                                                                 
2023-12-01 08:00:00.041508761-05:00  2023-12-01 13:00:00.041330717+00:00      0             1         260937      T    B      0  15924.50     1      0        15953  184385641   NQZ3            True               True
2023-12-01 08:00:00.152503179-05:00  2023-12-01 13:00:00.152250041+00:00      0             1         260937      T    B      0  15924.75     2      0        15875  184385741   NQZ3            True               True
2023-12-01 08:00:00.152578527-05:00  2023-12-01 13:00:00.

Just outputting the current 5-minute OHLC dataframe for verification/troubleshooting.

In [12]:
# Output the current 5-minute DataFrame to csv for troubleshooting
df_5m.to_csv(f'{temp_data_dir}/{symbol}_5min_sniper_trade_results.csv')
print(f"Current 5-minute DataFrame saved to .csv at '{temp_data_dir}/{symbol}_5min_sniper_trade_results.csv' \n\n") 

with pd.option_context("display.min_rows", 10, "display.max_columns", None, "display.width", 1000):
    print(df_5m)

Current 5-minute DataFrame saved to .csv at '../strategies/data/30s_sniper_base_model/NQZ3_5min_sniper_trade_results.csv' 


                               open      high       low     close  is_data_window  SwingHigh  SwingLow  BreakHigh  BreakLow  DownClose  UpClose  is_trading_window
ts_recv                                                                                                                                                           
2023-12-01 08:00:00-05:00  15924.50  15940.50  15923.00  15935.75            True        NaN       NaN      False     False      False    False               True
2023-12-01 08:05:00-05:00  15936.00  15941.50  15935.00  15937.50            True        NaN       NaN      False     False      False    False               True
2023-12-01 08:10:00-05:00  15937.50  15942.50  15929.50  15932.75            True        NaN       NaN      False     False      False    False               True
2023-12-01 08:15:00-05:00  15932.50  15941.00  15929.75  159

***The code below is a work in progress.***

This is code that I started with in the original iteration and have been refactoring as I go. Assume that this is nonsense, is not working correctly, and currently invalid.

In [13]:
########################################################################
# The code below is a work in progress
########################################################################

# Check for Bullish and Bearish Fair Value Gaps (FVG) using the most granular data (1-minute, currently)
df_30s["BullFVG"] = (df_30s["low"].shift(-1).where(df_5m["UpClose"].shift(1) == True) > df_30s["high"].shift(1).where(df_5m["UpClose"].shift(1) == True)).ffill()
df_30s["BearFVG"] = (df_30s["high"].shift(-1).where(df_5m["DownClose"].shift(1) == True) < df_30s["low"].shift(1).where(df_5m["DownClose"].shift(1) == True)).ffill()

# When a Fair Value Gap is created, note the entry price
df_30s["LongEntry"] = (df_30s["low"].shift(-1).where(df_30s["BullFVG"] == True)).shift(1)
df_30s["ShortEntry"] = (df_30s["high"].shift(-1).where(df_30s["BearFVG"] == True)).shift(1)

# Determine stop loss and profit target prices and add them to the dataframe
df_30s["LongStopLoss"] = (df_30s["LongEntry"] - stop_loss)
df_30s["LongProfitTarget"] = (df_30s["LongEntry"] + profit_target)
df_30s["ShortStopLoss"] = (df_30s["ShortEntry"] + stop_loss)
df_30s["ShortProfitTarget"] = (df_30s["ShortEntry"] - profit_target)

# Output the DataFrame with swing high, swing low, BreakHigh, BreakLow, DownClose, and UpClose signals
with pd.option_context("display.max_rows", 20, "display.max_columns", None, "display.width", 1000):
    print(df_30s)
    
# Output the 30-second results to .csv
df_30s.to_csv(f"{temp_data_dir}/{symbol}_30s_sniper_trade_results.csv")


                               open      high       low     close  is_data_window  is_trading_window  BullFVG  BearFVG  LongEntry  ShortEntry  LongStopLoss  LongProfitTarget  ShortStopLoss  ShortProfitTarget
ts_recv                                                                                                                                                                                                        
2023-12-01 08:00:00-05:00  15924.50  15926.00  15923.00  15923.75            True               True    False    False        NaN         NaN           NaN               NaN            NaN                NaN
2023-12-01 08:00:30-05:00  15924.75  15929.00  15924.75  15928.25            True               True    False    False        NaN         NaN           NaN               NaN            NaN                NaN
2023-12-01 08:01:00-05:00  15928.00  15931.25  15928.00  15931.00            True               True    False    False        NaN         NaN           NaN             

#### Need to add backtesting library functions and code below. The following rules should be included:

* Trade Window: 08:00AM - 16:00PM EDT (completed)
* No new trade executions between 09:25AM - 09:35AM; open positions are fine to hold through the New York open at 09:30AM
* No new trades executed after 15:55PM, open positions are find to hold through the New York close at 16:00PM
* All positions should be closed by 16:09:59PM; this could be extended based on funding company or broker trading windows
* While a position is open, new trade setups and executions may be added in the same direction
* While a position is open, oppositing setups should be ignored, but still tracked in case the actual entry takes place after the open position closes
* Rules regarding news and economic numbers release times should be added ASAP once the historical results are generating successfully